Import necessary modules:

In [ ]:
import numpy as np
import pandas as pd
import datetime

Assuming all data was downloaded using ``Clean_Energy

In [ ]:
def clean_solar_generation_data():
    entire_df = None
    for year in list(range(2013,2017)):
        entire_df = export_year_data_2013_to_2016(year, big_df=entire_df)
    for year in list(range(2017,2020)):
        entire_df = export_year_data_2017_to_2019(year, big_df=entire_df)
    entire_df.to_csv('cleaned_solar_generation_data.csv', sep='\t', index=False)
    return    

In [ ]:
def export_year_data_2017_to_2019(year, big_df=None):
    year_df=None
    for month in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']:
        year_df = export_month_data_2017_to_2019(month, year, df= year_df)
    if big_df is None:
        return year_df
    else:
        return big_df.append(year_df)

In [ ]:
def export_month_data_2017_to_2019(month, year, df=None):
    assert year in [2017, 2018, 2019], 'Not the correct function to clean this year - '+str(year)+' data'
    filename = 'IntGenbyFuel'+str(year)+'.xlsx'
    dummy_df = pd.read_excel(filename, sheet_name=month)
    select_solar = dummy_df['Fuel']=='Solar'
    solar_df = dummy_df[select_solar]
    solar_df.reset_index(drop=True, inplace=True)
    #Select only date and 30' time stamp as necessary column indexes:
    cols = [c for c in solar_df.columns if (c.lower()[-2:]=='30' or c.lower()[-2:]=='00')]
    cols = ['Date'] + cols
    solar_df=solar_df[cols]
    rows_no, columns_no = solar_df.shape
    result_df = pd.DataFrame(columns = ['Date','hhmm','Solar'])
    # Flatten solar_df information which has each date as one row and hour&min as columns into result_df that has date
    for r in range(rows_no):
        for c in range(1,columns_no):
            formated_date = solar_df.iloc[r,0].strftime("%m/%d/%Y")
            result_df = result_df.append({'Date': formated_date,'hhmm':solar_df.columns[c],'Solar':solar_df.iloc[r,c]},ignore_index=True)
    if df is None:
        return result_df
    else:
        return df.append(result_df)

In [ ]:
def export_year_data_2013_to_2016(year, big_df=None):
    year_df=None
    for month in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']:
        year_df = export_month_data_2013_to_2016(month, year, df= year_df)
    if big_df is None:
        return year_df
    else:
        return big_df.append(year_df)

In [ ]:
def export_month_data_2013_to_2016(month, year, df=None):
    try:
        filename = 'IntGenByFuel'+str(year)+'.xlsx'
        dummy_df = pd.read_excel(filename, sheet_name=month+str(year)[-2:])
    except FileNotFoundError:
        filename = 'IntGenByFuel'+str(year)+'.xls'
        dummy_df = pd.read_excel(filename, sheet_name=month+str(year)[-2:])
    select_solar = dummy_df['Date-Fuel'].str.endswith('Solar')
    solar_df = dummy_df[select_solar]
    solar_df.reset_index(drop=True, inplace=True)
    # update columns labels in case it was in datetime time format:
    column_label = []
    for c in solar_df.columns:
        if isinstance(c, datetime.time):
            update_c = c.strftime("%H:%M")
            column_label.append(update_c)
        elif isinstance(c, datetime.datetime):
            update_c = c.strftime("%H:%M")
            column_label.append(update_c)
        else:
            column_label.append(c)
    solar_df.columns = column_label
    #Select only date and 30' time stamp as necessary column indexes:
    cols = [c for c in solar_df.columns if (c.lower()[-2:]=='30' or c.lower()[-2:]=='00')]
    cols = ['Date-Fuel'] + cols
    solar_df=solar_df[cols]
    rows_no, columns_no = solar_df.shape
    result_df = pd.DataFrame(columns = ['Date','hhmm','Solar'])
    # Flatten solar_df information which has each date as one row and hour&min as columns into result_df that has date
    for r in range(rows_no):
        for c in range(1,columns_no):
            string_of_date = solar_df.iloc[r,0][:8]
            formated_date = string_of_date[:6]+'20'+string_of_date[-2:]
            result_df = result_df.append({'Date':formated_date,'hhmm':solar_df.columns[c],'Solar':solar_df.iloc[r,c]},ignore_index=True)
    if df is None:
        return result_df
    else:
        return df.append(result_df)

In [ ]:
clean_solar_generation_data()